In [135]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torchviz import make_dot
from torchsummary import summary

In [136]:
# data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
# file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_combined.csv'
# train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_wav2vec.csv'


In [137]:
data_df=pd.read_csv(train_path, index_col=0)
# data_df=pd.read_csv(train_path)

In [138]:
display(data_df.head())

,0,1,2,3,4,5,6,7,8,9,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
track_id,,,,,,,,,,,,,,,,,,,,,
2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,4.041143,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,12.864646,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,3.136315,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,-2.599205,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,-8.300909,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk


In [139]:

X = StandardScaler().fit_transform(data_df.drop('genre', axis=1))
y = pd.factorize(data_df['genre'])[0]  # Convert genre to numerical labels


In [140]:
display(y)

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [141]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [142]:
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

X_train_tensor_rnn = X_train_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
X_test_tensor_rnn = X_test_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
train_dataset_rnn = TensorDataset(X_train_tensor_rnn, y_train_tensor)
test_dataset_rnn = TensorDataset(X_test_tensor_rnn, y_test_tensor)
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=128, shuffle=True)
test_loader_rnn = DataLoader(test_dataset_rnn, batch_size=128, shuffle=False)


In [143]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),

            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(128,out_features=output_dim)

        )
        
    def forward(self, x):
        return self.layers(x)


In [144]:
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN1D, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=6, padding=8),  # Adjust padding if necessary
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(32, 128, kernel_size=6, padding=8),  # Adjust padding if necessary
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        # Calculate size of the flattened output after conv and pooling layers
        self.output_size = self._get_conv_output(input_dim)

        self.fc_layers = nn.Sequential(
            nn.Linear(self.output_size, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
    def _get_conv_output(self, input_dim):
        # Dummy input for calculating size
        input = torch.zeros(1, 1, input_dim)
        output = self.conv_layers(input)
        n_size = output.data.view(1, -1).size(1)
        return n_size

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc_layers(x)


In [145]:
class RNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=512, num_layers=8):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x

class LSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=512, num_layers=8):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, (hn, cn) = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x


In [146]:
class EnsembleModel(nn.Module):
    def __init__(self, mlp, cnn, rnn, lstm, output_dim):
        super(EnsembleModel, self).__init__()
        self.mlp = mlp
        self.cnn = cnn
        self.rnn = rnn
        self.lstm = lstm
        self.fc = nn.Linear(output_dim * 4, output_dim)  # Assuming each model outputs the class probabilities
    
    def forward(self, x):
        out_mlp = self.mlp(x)
        out_cnn = self.cnn(x)
        out_rnn = self.rnn(x.unsqueeze(1))  # Assume RNN/LSTM expects sequence data
        out_lstm = self.lstm(x.unsqueeze(1))
        
        # Concatenate along the feature dimension
        combined = torch.cat((out_mlp, out_cnn, out_rnn, out_lstm), dim=1)
        output = self.fc(combined)
        return output


In [147]:
def train_model(model, train_loader, test_loader, device, epochs=100):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    acc=0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        total = correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc= max(acc,100 * correct / total)

        print(f'Epoch {epoch+1}: Accuracy: {100 * correct / total}%')
    return(acc)

In [148]:
input_size = X_train.shape[1]
num_classes = len(np.unique(y))

In [149]:
model_mlp=MLP(input_size,num_classes).to(device)
train_model(model_mlp, train_loader, test_loader, device)
# 54.127063531765884 entropy loss

Epoch 1: Accuracy: 33.48820586132952%
Epoch 2: Accuracy: 38.92065761258041%
Epoch 3: Accuracy: 39.09935668334525%
Epoch 4: Accuracy: 42.53037884203002%
Epoch 5: Accuracy: 44.603288062902074%
Epoch 6: Accuracy: 42.17298070050036%
Epoch 7: Accuracy: 45.38956397426733%
Epoch 8: Accuracy: 45.38956397426733%
Epoch 9: Accuracy: 45.06790564689064%
Epoch 10: Accuracy: 47.53395282344532%
Epoch 11: Accuracy: 45.103645461043605%
Epoch 12: Accuracy: 46.31879914224446%
Epoch 13: Accuracy: 48.391708363116514%
Epoch 14: Accuracy: 45.103645461043605%
Epoch 15: Accuracy: 48.85632594710508%
Epoch 16: Accuracy: 43.13795568263045%
Epoch 17: Accuracy: 46.35453895639743%
Epoch 18: Accuracy: 47.60543245175125%
Epoch 19: Accuracy: 47.60543245175125%
Epoch 20: Accuracy: 46.24731951393853%
Epoch 21: Accuracy: 50.17869907076483%
Epoch 22: Accuracy: 47.49821300929235%
Epoch 23: Accuracy: 44.85346676197284%
Epoch 24: Accuracy: 47.81987133666905%
Epoch 25: Accuracy: 49.17798427448177%
Epoch 26: Accuracy: 48.7133666

51.78699070764832

In [150]:


# Assuming X_train.shape[1] is defined and device is defined
dummy_input = torch.zeros(1,X_train.shape[1]).to(device)  # Move dummy_input to the same device as the model
model_mlp.to(device)

# Visualize the model using torchviz
model_vis = make_dot(model_mlp(dummy_input), params=dict(model_mlp.named_parameters()))
model_vis.render("neural_network_visualization_mlp", format="png", view=True)

# Assuming num_classes is defined
summary(model_mlp, input_size=dummy_input.size()[1:])  # Correctly specifying input_size as a tuple


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                 [-1, 1185]           2,370
            Linear-2                 [-1, 2048]       2,428,928
         LeakyReLU-3                 [-1, 2048]               0
           Dropout-4                 [-1, 2048]               0
            Linear-5                 [-1, 1024]       2,098,176
         LeakyReLU-6                 [-1, 1024]               0
           Dropout-7                 [-1, 1024]               0
            Linear-8                  [-1, 512]         524,800
       BatchNorm1d-9                  [-1, 512]           1,024
        LeakyReLU-10                  [-1, 512]               0
          Dropout-11                  [-1, 512]               0
           Linear-12                  [-1, 256]         131,328
      BatchNorm1d-13                  [-1, 256]             512
        LeakyReLU-14                  [

In [151]:
model_cnn=CNN1D(input_size,num_classes).to(device)
train_model(model_cnn, train_loader, test_loader, device) 
# 51.52576288144072 entropy loss

Epoch 1: Accuracy: 42.35167977126519%
Epoch 2: Accuracy: 46.42601858470336%
Epoch 3: Accuracy: 49.7855611150822%
Epoch 4: Accuracy: 49.82130092923517%
Epoch 5: Accuracy: 48.74910650464618%
Epoch 6: Accuracy: 50.6433166547534%
Epoch 7: Accuracy: 49.28520371694067%
Epoch 8: Accuracy: 50.7505360972123%
Epoch 9: Accuracy: 50.964974982130094%
Epoch 10: Accuracy: 49.42816297355254%
Epoch 11: Accuracy: 49.571122230164406%
Epoch 12: Accuracy: 49.82130092923517%
Epoch 13: Accuracy: 50.60757684060043%
Epoch 14: Accuracy: 50.428877769835594%
Epoch 15: Accuracy: 50.17869907076483%
Epoch 16: Accuracy: 50.929235167977126%
Epoch 17: Accuracy: 50.71479628305933%
Epoch 18: Accuracy: 50.89349535382416%
Epoch 19: Accuracy: 48.71336669049321%
Epoch 20: Accuracy: 50.321658327376696%
Epoch 21: Accuracy: 49.3566833452466%
Epoch 22: Accuracy: 50.60757684060043%
Epoch 23: Accuracy: 50.357398141529664%
Epoch 24: Accuracy: 50.929235167977126%
Epoch 25: Accuracy: 51.036454610436024%
Epoch 26: Accuracy: 49.2852037

51.17941386704789

In [152]:


# Assuming X_train.shape[1] is defined and device is defined
dummy_input = torch.zeros(1,X_train.shape[1]).to(device)  # Move dummy_input to the same device as the model
model_cnn.to(device)

# Visualize the model using torchviz
model_vis = make_dot(model_cnn(dummy_input), params=dict(model_cnn.named_parameters()))
model_vis.render("neural_network_visualization_cnn", format="png", view=True)

# Assuming num_classes is defined
summary(model_cnn, input_size=dummy_input.size()[1:])  # Correctly specifying input_size as a tuple


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 32, 1196]             224
              ReLU-2             [-1, 32, 1196]               0
         MaxPool1d-3              [-1, 32, 598]               0
            Conv1d-4             [-1, 128, 609]          24,704
              ReLU-5             [-1, 128, 609]               0
         MaxPool1d-6             [-1, 128, 304]               0
            Linear-7                  [-1, 128]       4,980,864
              ReLU-8                  [-1, 128]               0
            Linear-9                    [-1, 8]           1,032
Total params: 5,006,824
Trainable params: 5,006,824
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 2.22
Params size (MB): 19.10
Estimated Total Size (MB): 21.32
-------------------------------------

In [153]:
model_rnn=RNN(input_size,num_classes).to(device)
train_model(model_rnn, train_loader_rnn, test_loader_rnn, device) 
#50.17508754377189 entropy loss

Epoch 1: Accuracy: 17.512508934953537%
Epoch 2: Accuracy: 18.90636168691923%
Epoch 3: Accuracy: 20.943531093638313%
Epoch 4: Accuracy: 20.514653323802715%
Epoch 5: Accuracy: 25.23230879199428%
Epoch 6: Accuracy: 20.97927090779128%
Epoch 7: Accuracy: 19.656897784131523%
Epoch 8: Accuracy: 22.980700500357397%
Epoch 9: Accuracy: 22.8734810578985%
Epoch 10: Accuracy: 19.835596854896355%
Epoch 11: Accuracy: 25.62544674767691%
Epoch 12: Accuracy: 23.33809864188706%
Epoch 13: Accuracy: 26.018584703359544%
Epoch 14: Accuracy: 25.589706933523946%
Epoch 15: Accuracy: 27.1265189421015%
Epoch 16: Accuracy: 27.019299499642603%
Epoch 17: Accuracy: 26.018584703359544%
Epoch 18: Accuracy: 22.980700500357397%
Epoch 19: Accuracy: 26.733380986418872%
Epoch 20: Accuracy: 27.412437455325232%
Epoch 21: Accuracy: 27.269478198713365%
Epoch 22: Accuracy: 27.269478198713365%
Epoch 23: Accuracy: 24.731951393852754%
Epoch 24: Accuracy: 25.804145818441743%
Epoch 25: Accuracy: 26.23302358827734%
Epoch 26: Accuracy:

27.412437455325232

In [154]:


# Assuming X_train.shape[1] is defined and device is defined
dummy_input = torch.zeros(1,1,X_train.shape[1]).to(device)  # Move dummy_input to the same device as the model
model_rnn.to(device)

# Visualize the model using torchviz
model_vis = make_dot(model_rnn(dummy_input), params=dict(model_rnn.named_parameters()))
model_vis.render("neural_network_visualization_rnn", format="png", view=True)

# Assuming num_classes is defined
summary(model_rnn, input_size=dummy_input.size()[1:])  # Correctly specifying input_size as a tuple


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               RNN-1  [[-1, 1, 512], [-1, 2, 512]]               0
            Linear-2                    [-1, 8]           4,104
Total params: 4,104
Trainable params: 4,104
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 4.00
Params size (MB): 0.02
Estimated Total Size (MB): 4.02
----------------------------------------------------------------


In [155]:
model_lstm=LSTM(input_size,num_classes).to(device)
train_model(model_lstm, train_loader_rnn, test_loader_rnn, device) 
#49.67483741870935 entropy loss

Epoch 1: Accuracy: 25.33952823445318%
Epoch 2: Accuracy: 29.949964260185848%
Epoch 3: Accuracy: 26.55468191565404%
Epoch 4: Accuracy: 30.164403145103645%
Epoch 5: Accuracy: 33.20228734810579%
Epoch 6: Accuracy: 33.30950679056469%
Epoch 7: Accuracy: 35.025017869907074%
Epoch 8: Accuracy: 35.38241601143674%
Epoch 9: Accuracy: 31.12937812723374%
Epoch 10: Accuracy: 33.84560400285918%
Epoch 11: Accuracy: 35.38241601143674%
Epoch 12: Accuracy: 34.73909935668335%
Epoch 13: Accuracy: 37.84846318799142%
Epoch 14: Accuracy: 38.777698355968546%
Epoch 15: Accuracy: 37.63402430307362%
Epoch 16: Accuracy: 37.77698355968549%
Epoch 17: Accuracy: 43.388134381701214%
Epoch 18: Accuracy: 37.27662616154396%
Epoch 19: Accuracy: 41.85132237312366%
Epoch 20: Accuracy: 42.74481772694782%
Epoch 21: Accuracy: 42.95925661186562%
Epoch 22: Accuracy: 42.99499642601859%
Epoch 23: Accuracy: 42.74481772694782%
Epoch 24: Accuracy: 39.421015010721945%
Epoch 25: Accuracy: 41.10078627591137%
Epoch 26: Accuracy: 43.10221

45.5325232308792

In [156]:


# Assuming X_train.shape[1] is defined and device is defined
dummy_input = torch.zeros(1,1,X_train.shape[1]).to(device)  # Move dummy_input to the same device as the model
model_lstm.to(device)

# Visualize the model using torchviz
model_vis = make_dot(model_lstm(dummy_input), params=dict(model_lstm.named_parameters()))
model_vis.render("neural_network_visualization_lstm", format="png", view=True)

# Assuming num_classes is defined
summary(model_lstm, input_size=dummy_input.size()[1:])  # Correctly specifying input_size as a tuple


AttributeError: 'tuple' object has no attribute 'size'

In [166]:
model_cnn_2=CNN1D(input_size,num_classes).to(device)
model_mlp_2=MLP(input_size,num_classes).to(device)
model_rnn_2=RNN(input_size,num_classes).to(device)
model_lstm_2=LSTM(input_size,num_classes).to(device)

model_ens=EnsembleModel(mlp=model_mlp_2,cnn=model_cnn_2,rnn=model_rnn_2,lstm=model_lstm_2,output_dim=num_classes).to(device)
train_model(model_ens, train_loader, test_loader, device)
# 53.62681340670335 accuracy entropy loss

Epoch 1: Accuracy: 40.35025017869907%
Epoch 2: Accuracy: 43.53109363831308%
Epoch 3: Accuracy: 44.674767691208004%
Epoch 4: Accuracy: 46.60471765546819%
Epoch 5: Accuracy: 48.14152966404575%
Epoch 6: Accuracy: 48.999285203716944%
Epoch 7: Accuracy: 47.53395282344532%
Epoch 8: Accuracy: 49.7855611150822%
Epoch 9: Accuracy: 47.56969263759829%
Epoch 10: Accuracy: 50.285918513223734%
Epoch 11: Accuracy: 49.2494639027877%
Epoch 12: Accuracy: 48.284488920657616%
Epoch 13: Accuracy: 48.49892780557541%
Epoch 14: Accuracy: 50.17869907076483%
Epoch 15: Accuracy: 48.32022873481058%
Epoch 16: Accuracy: 49.8927805575411%
Epoch 17: Accuracy: 50.17869907076483%
Epoch 18: Accuracy: 48.82058613295211%
Epoch 19: Accuracy: 48.60614724803431%
Epoch 20: Accuracy: 49.070764832022874%
Epoch 21: Accuracy: 48.89206576125804%
Epoch 22: Accuracy: 49.070764832022874%
Epoch 23: Accuracy: 47.784131522516084%
Epoch 24: Accuracy: 48.67762687634024%
Epoch 25: Accuracy: 49.42816297355254%
Epoch 26: Accuracy: 49.2494639

50.428877769835594

In [ ]:


# Assuming X_train.shape[1] is defined and device is defined
dummy_input = torch.zeros(1,X_train.shape[1]).to(device)  # Move dummy_input to the same device as the model
model_ens.to(device)

# Visualize the model using torchviz
model_vis = make_dot(model_ens(dummy_input), params=dict(model_ens.named_parameters()))
model_vis.render("neural_network_visualization_ens", format="png", view=True)

# Assuming num_classes is defined
summary(model_ens, input_size=(1,X_train.shape[1]))  # Correctly specifying input_size as a tuple


In [169]:
def model_summary(model):
    print(model)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'Total trainable parameters: {total_params}')


In [170]:
# Print model summary
model_summary(model_ens)

EnsembleModel(
  (mlp): MLP(
    (layers): Sequential(
      (0): BatchNorm1d(1185, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1185, out_features=2048, bias=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=2048, out_features=1024, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Dropout(p=0.1, inplace=False)
      (7): Linear(in_features=1024, out_features=512, bias=True)
      (8): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): LeakyReLU(negative_slope=0.01)
      (10): Dropout(p=0.1, inplace=False)
      (11): Linear(in_features=512, out_features=256, bias=True)
      (12): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (13): LeakyReLU(negative_slope=0.01)
      (14): Linear(in_features=256, out_features=128, bias=True)
      (15): BatchNorm1d(128, eps=1e-05, momentu

AttributeError: 'tuple' object has no attribute 'size'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random

input_size = X_train.shape[1]
output_size = len(np.unique(y))

In [ ]:
import torch
from torch.utils.data import DataLoader

def train_model(model, optimizer, train_loader, test_loader, device, epochs=50):
    criterion = torch.nn.CrossEntropyLoss()  # Assuming classification task
    best_acc = 0
    best_epoch = 0
    
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        total_train = 0
        correct_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train * 100
        
        # Validation phase
        model.eval()  # Set model to evaluate mode
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():  # No gradient calculation for validation
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(test_loader.dataset)
        val_accuracy = correct / total * 100
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%')
        
        # Save the model if the validation accuracy is the best we've seen so far.
        if val_accuracy > best_acc:
            best_acc = val_accuracy
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')  # Save best model
            
    print(f'Best Validation Accuracy: {best_acc:.2f}% at Epoch {best_epoch+1}')
    return best_acc


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR  # Learning rate scheduler

def train_model_best(model, optimizer, train_loader, test_loader, device, epochs=50):
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # Reduce LR every 30 epochs
    best_acc = 0
    best_epoch = 0
    patience = 10  # Early stopping patience
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_train = 0
        correct_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train * 100

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(test_loader.dataset)
        val_accuracy = correct / total * 100

        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%')

        # Learning rate scheduler step
        scheduler.step()

        # Early stopping check
        if val_accuracy > best_acc:
            best_acc = val_accuracy
            best_epoch = epoch
            patience_counter = 0  # Reset counter
            torch.save(model.state_dict(), 'best_model.pth')  # Save best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Stopping early at epoch {epoch + 1}")
                break

    print(f'Best Validation Accuracy: {best_acc:.2f}% at Epoch {best_epoch+1}')
    return best_acc


In [ ]:


class NeuralNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers, neurons_start, dropout_rate, use_bn, activation_fn):
        super(NeuralNet, self).__init__()
        layers = []

        # Calculate the decrement step to ensure the last layer has 64 neurons
        if neurons_start > 64:
            decrement = (neurons_start - 64) / (hidden_layers - 1)
        else:
            decrement = 0

        current_neurons = neurons_start

        for i in range(hidden_layers):
            next_neurons = max(64, int(current_neurons - decrement * i))  # Ensure we do not go below 64 neurons
            layers.append(nn.Linear(input_dim, next_neurons))
            if use_bn:
                layers.append(nn.BatchNorm1d(next_neurons))
            layers.append(activation_fn())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = next_neurons

        layers.append(nn.Linear(next_neurons, output_dim))  # Final layer to output
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [ ]:
params = {
    'num_layers': [2, 4, 6,8],  # Number of hidden layers
    'neurons_start': [256, 512, 1024,2048],  # Starting number of neurons
    'dropout_rate': [0.05, 0.1, 0.2],
    'batch_size': [64, 128, 256],
    'use_bn': [True, False],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'optimizer': [torch.optim.Adam, torch.optim.SGD],
    'activation_fn': [nn.ReLU, nn.LeakyReLU, nn.ELU],

}

# Random Search
num_searches = 256  # Number of random configurations to try
best_acc = 0
best_params = None
best_model = None

In [ ]:
for _ in range(num_searches):  # Number of configurations to try
    chosen_params = {k: random.choice(v) for k, v in params.items()}
    
    # Create model
    model = NeuralNet(input_size, num_classes, chosen_params['num_layers'], chosen_params['neurons_start'],
                      chosen_params['dropout_rate'], chosen_params['use_bn'], chosen_params['activation_fn']).to(device)
    
    # Setup optimizer
    optimizer = chosen_params['optimizer'](model.parameters(), lr=chosen_params['learning_rate'])
    
    # Train the model (function to be defined based on your setup)
    acc = train_model(model, optimizer, train_loader, test_loader, device)
    # acc= train_model_best(model, optimizer, train_loader, test_loader, device)
    # Track the best model
    if acc > best_acc:
        best_acc = acc
        best_params = chosen_params
        best_model = model_mlp

Epoch 1/50:
Train Loss: 2.1419 - Train Acc: 12.37%
Val Loss: 2.0727 - Val Acc: 17.08%
Epoch 2/50:
Train Loss: 2.1050 - Train Acc: 14.09%
Val Loss: 2.0481 - Val Acc: 18.87%
Epoch 3/50:
Train Loss: 2.0598 - Train Acc: 17.47%
Val Loss: 2.0239 - Val Acc: 20.26%
Epoch 4/50:
Train Loss: 2.0501 - Train Acc: 19.21%
Val Loss: 2.0111 - Val Acc: 20.94%
Epoch 5/50:
Train Loss: 2.0328 - Train Acc: 19.67%
Val Loss: 1.9887 - Val Acc: 23.02%
Epoch 6/50:
Train Loss: 2.0219 - Train Acc: 20.79%
Val Loss: 1.9745 - Val Acc: 24.12%
Epoch 7/50:
Train Loss: 2.0067 - Train Acc: 21.73%
Val Loss: 1.9574 - Val Acc: 25.91%
Epoch 8/50:
Train Loss: 1.9901 - Train Acc: 22.98%
Val Loss: 1.9521 - Val Acc: 28.27%
Epoch 9/50:
Train Loss: 1.9852 - Train Acc: 22.98%
Val Loss: 1.9388 - Val Acc: 29.56%
Epoch 10/50:
Train Loss: 1.9711 - Train Acc: 24.44%
Val Loss: 1.9300 - Val Acc: 30.20%
Epoch 11/50:
Train Loss: 1.9537 - Train Acc: 26.21%
Val Loss: 1.9234 - Val Acc: 31.56%
Epoch 12/50:
Train Loss: 1.9534 - Train Acc: 26.25%


KeyboardInterrupt: 

In [ ]:
print(f"Best Accuracy: {best_acc}")
print(f"Best Parameters: {best_params}")
# Best Accuracy: 54.17708854427213
# Best Parameters: {'num_layers': 4, 'neurons_start': 2048, 'dropout_rate': 0.1, 'batch_size': 128, 'use_bn': True, 'learning_rate': 0.0001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


Best Accuracy: 54.17708854427213
Best Parameters: {'num_layers': 4, 'neurons_start': 2048, 'dropout_rate': 0.1, 'batch_size': 128, 'use_bn': True, 'learning_rate': 0.0001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


In [ ]:
for _ in range(num_searches):  # Number of configurations to try
    chosen_params = {k: random.choice(v) for k, v in params.items()}
    
    # Create model
    model = NeuralNet(input_size, num_classes, chosen_params['num_layers'], chosen_params['neurons_start'],
                      chosen_params['dropout_rate'], chosen_params['use_bn'], chosen_params['activation_fn']).to(device)
    
    # Setup optimizer
    optimizer = chosen_params['optimizer'](model.parameters(), lr=chosen_params['learning_rate'])
    
    # Train the model (function to be defined based on your setup)
    # acc = train_model(model, optimizer, train_loader, test_loader, device)
    acc= train_model_best(model, optimizer, train_loader, test_loader, device)
    # Track the best model
    if acc > best_acc:
        best_acc = acc
        best_params = chosen_params
        best_model = model_mlp

Epoch 1/50:
Train Loss: 1.9873 - Train Acc: 23.59%
Val Loss: 1.9567 - Val Acc: 30.47%
Epoch 2/50:
Train Loss: 1.7650 - Train Acc: 36.00%
Val Loss: 1.7266 - Val Acc: 38.17%
Epoch 3/50:
Train Loss: 1.6560 - Train Acc: 41.85%
Val Loss: 1.6096 - Val Acc: 43.42%
Epoch 4/50:
Train Loss: 1.5678 - Train Acc: 45.87%
Val Loss: 1.5538 - Val Acc: 45.62%
Epoch 5/50:
Train Loss: 1.4977 - Train Acc: 49.21%
Val Loss: 1.5740 - Val Acc: 44.67%
Epoch 6/50:
Train Loss: 1.4261 - Train Acc: 52.43%
Val Loss: 1.5148 - Val Acc: 46.62%
Epoch 7/50:
Train Loss: 1.3709 - Train Acc: 54.03%
Val Loss: 1.5060 - Val Acc: 47.37%
Epoch 8/50:
Train Loss: 1.3169 - Train Acc: 56.83%
Val Loss: 1.5024 - Val Acc: 47.02%
Epoch 9/50:
Train Loss: 1.2744 - Train Acc: 58.72%
Val Loss: 1.5217 - Val Acc: 46.57%
Epoch 10/50:
Train Loss: 1.2267 - Train Acc: 60.67%
Val Loss: 1.4574 - Val Acc: 49.12%
Epoch 11/50:
Train Loss: 1.1793 - Train Acc: 62.35%
Val Loss: 1.4602 - Val Acc: 49.32%
Epoch 12/50:
Train Loss: 1.1238 - Train Acc: 64.34%


In [ ]:

print(f"Best Accuracy: {best_acc}")
print(f"Best Parameters: {best_params}")

# Best Accuracy: 54.82741370685342
# Best Parameters: {'num_layers': 2, 'neurons_start': 512, 'dropout_rate': 0.1, 'batch_size': 64, 'use_bn': False, 'learning_rate': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


Best Accuracy: 54.82741370685342
Best Parameters: {'num_layers': 2, 'neurons_start': 512, 'dropout_rate': 0.1, 'batch_size': 64, 'use_bn': False, 'learning_rate': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import randint

# # Setup the parameter grid
# param_dist_knn = {
#     'n_neighbors': randint(1, 30),  # Increased range for number of neighbors
#     'weights': ['uniform', 'distance'],  # Weight functions
#     'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],  # Distance metrics
#     'p': randint(1, 5)  # Power parameter for the Minkowski metric
# }
# # Initialize the KNN model
# knn = KNeighborsClassifier()

# # Setup RandomizedSearchCV
# random_search_knn = RandomizedSearchCV(knn, param_distributions=param_dist_knn, n_iter=25, cv=5, verbose=1, random_state=42, n_jobs=8)

# # Fit the model
# random_search_knn.fit(X_train, y_train)

# # Best parameters and score
# print("Best parameters found for KNN:", random_search_knn.best_params_)
# print("Best cross-validated score achieved for KNN:", random_search_knn.best_score_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for KNN: {'metric': 'manhattan', 'n_neighbors': 8, 'p': 4, 'weights': 'distance'}
Best cross-validated score achieved for KNN: 0.42035029190992496


In [ ]:
from sklearn.metrics import accuracy_score

# Use the best estimator directly from the RandomizedSearchCV results for KNN
# best_knn = random_search_knn.best_estimator_
best_knn= {'metric': 'manhattan', 'n_neighbors': 8, 'p': 4, 'weights': 'distance'}


# Predict on the test set
y_pred_knn = best_knn.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Model Test Accuracy: {accuracy_knn * 100:.2f}%")


KNN Model Test Accuracy: 43.02%


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# For KNN
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))
print("\nKNN Classification Report:\n", classification_report(y_test, y_pred_knn))
